In [1]:
from sklearn.datasets import load_breast_cancer
from pycaret.anomaly import *
from pycaret.datasets import get_data
import mlflow

/home/ppleeqq/.local/lib/python3.6/site-packages/pycaret/loggers/mlflow_logger.py:14: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  import mlflow


In [2]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


sns.set(style="whitegrid")
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

Using TensorFlow backend.


cuda


### 시드 고정

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    ## 위에서 random.seed 지정해줬는데 왜 np.random 지정해야하는지 모르겠음!!
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(200)

### 변수선택으로 dataset 만들기

In [27]:
train_df = pd.read_csv('./input/train.csv')
train_df = train_df.drop(columns=['ID',"V13","V15","V19","V20","V22","V23","V24","V25","V26","V28","V29","V30"])

val_df = pd.read_csv('./input/val.csv')
val_c=val_df['Class']
val_df = val_df.drop(columns=['ID','Class',"V13","V15","V19","V20","V22","V23","V24","V25","V26","V28","V29","V30"])


In [10]:
#data setup
anom=setup(data=val_df,silent=True)

,Description,Value
0,session_id,3483
1,Original Data,"(28462, 18)"
2,Missing Values,False
3,Numeric Features,18
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(28462, 18)"
9,CPU Jobs,-1


- 이하 승우가 했던 것처럼 변수선택해서 다시 돌리기
> import를 anomaly랑 classification 둘 다 했더니 classification인줄 알고 오류가 났다. anomaly만 해야할듯.

- ‘abod’      | Angle-base Outlier Detection      |
- | ‘iforest’   | Isolation Forest                  |
- | ‘cluster’   | Clustering-Based Local Outlier    |
- | ‘cof’       | Connectivity-Based Outlier Factor |
- | ‘histogram’ | Histogram-based Outlier Detection |
- | ‘knn’       | k-Nearest Neighbors Detector      |
- | ‘lof’       | Local Outlier Factor              |
- | ‘svm’       | One-class SVM detector            |
- | ‘pca’       | Principal Component Analysis      |
- | ‘mcd’       | Minimum Covariance Determinant    |
- | ‘sod’       | Subspace Outlier Detection        | > 너무 오래걸려서 패스
- | ‘sos        | Stochastic Outlier Selection 

### 1-1.Isolation forest

In [38]:
#Train the anomaly detection model
anom_model = create_model(model = 'iforest', fraction = 0.001)

In [39]:
#score the dataset
results=assign_model(anom_model)
val_pred=results['Anomaly']
print(val_pred.value_counts())

0    28433
1       29
Name: Anomaly, dtype: int64


In [40]:
#score
val_score = f1_score(val_c, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_c, val_pred))

Validation F1 Score : [0.7200488221434351]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.45      0.43      0.44        30

    accuracy                           1.00     28462
   macro avg       0.72      0.72      0.72     28462
weighted avg       1.00      1.00      1.00     28462



- 원래 score가 0.7030820840915222 였는데 0.02정도 올랐다.
### test 데이터 제출

In [5]:
test_df = pd.read_csv('./input/test.csv') # Train
test_df.head()
test_x = test_df.drop(columns=['ID',"V13","V15","V19","V20","V22","V23","V24","V25","V26","V28","V29","V30"])

In [6]:
#data setup
anom=setup(data=test_x,silent=True)

,Description,Value
0,session_id,897
1,Original Data,"(142503, 18)"
2,Missing Values,False
3,Numeric Features,18
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(142503, 18)"
9,CPU Jobs,-1


In [7]:
anom_model = create_model(model = 'iforest', fraction = 0.001)
results=assign_model(anom_model)
test_pred=results['Anomaly']

In [9]:
 # model prediction
submit = pd.read_csv('./input/sample_submission.csv')
submit['Class'] = test_pred
submit.to_csv('./submit.csv', index=False)